# Lesson 4: Constructing Trace Polynomials

In this lesson, we'll examine how trace data is transformed into polynomials. This is a key step in creating a STARK, as it allows us to work with trace data in algebraic form.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.polynomial import polynomial as P
from scipy.interpolate import lagrange

def create_trace_polynomial(values, domain):
    """Creates a polynomial through Lagrange interpolation.
    
    Args:
        values (np.array): Trace values
        domain (np.array): Domain points
        
    Returns:
        np.poly1d: Interpolation polynomial
    """
    return lagrange(domain, values)

## Creating Polynomials for our Fibonacci Trace

In [ ]:
# Import functions from previous lessons
from lesson_1_execution_trace import create_fibonacci_trace
from lesson_3_padding_trace import pad_trace_to_power_of_two

# Create and pad trace
trace = create_fibonacci_trace(3, 5)
padded_trace = pad_trace_to_power_of_two(trace)

# Create domain for interpolation
n_points = len(padded_trace)
domain = np.linspace(-1, 1, n_points)

# Create polynomials for each column
polynomials = {}
for column in padded_trace.columns[:3]:  # Only for registers
    values = padded_trace[column].values
    poly = create_trace_polynomial(values, domain)
    polynomials[column] = poly

# Visualization
plt.figure(figsize=(15, 5))

# Plot original points and polynomials
x_fine = np.linspace(-1, 1, 1000)
for column, poly in polynomials.items():
    plt.plot(domain, padded_trace[column], 'o', label=f'{column} (points)')
    plt.plot(x_fine, poly(x_fine), '-', label=f'{column} (polynomial)')

plt.title('Trace Polynomials')
plt.xlabel('Domain')
plt.ylabel('Value')
plt.grid(True)
plt.legend()
plt.show()

## Verifying Polynomial Properties

In [ ]:
def verify_polynomial_properties(poly, values, domain, name):
    """Verifies properties of a polynomial.
    
    Args:
        poly: Polynomial
        values: Original values
        domain: Domain points
        name: Polynomial name
    """
    # Check that polynomial passes through all points
    interpolation_error = np.max(np.abs(poly(domain) - values))
    print(f"{name}:")
    print(f"Maximum interpolation error: {interpolation_error:.2e}")
    print(f"Polynomial degree: {len(poly.coef) - 1}\n")

# Check each polynomial
for column, poly in polynomials.items():
    values = padded_trace[column].values
    verify_polynomial_properties(poly, values, domain, column)

## Important Notes

1. **Polynomial Degrees**:
   - Each polynomial's degree is at most n-1, where n is the padded trace length
   - This is important for subsequent operations' efficiency

2. **Interpolation Accuracy**:
   - Polynomials exactly pass through all trace points
   - This ensures all execution information is preserved

3. **Use in STARK**:
   - Polynomial representation enables efficient constraint checking
   - Facilitates application of cryptographic primitives

In the next lesson, we'll examine how these polynomials are used to create ZK commitments.